In [2]:
import numpy as np
import pandas as pd
import numpy as np
import os 
import time
from google.cloud import bigquery
import pandas_gbq

In [3]:
client = bigquery.Client()

In [5]:
query = '''SELECT 
  ano,
  sigla_uf,
  esfera,
  poderes, 
  quantidade_vinculos
FROM (
  SELECT 
    ano,
    sigla_uf,
    CASE 
      WHEN natureza_juridica IN ('1015','1040', '1074', '1104','1139', '1163', '1252', '1287', '1317', '1341') THEN 'Federal'
      WHEN natureza_juridica IN ('1023','1058', '1082', '1112', '1147', '1171', '1236','1260',  '1295', '1325') THEN 'Estadual'
      WHEN natureza_juridica IN ('1031','1066', '1120', '1155', '1180', '1244', '1279', '1309', '1333') THEN 'Municipal'
      ELSE 'Outros' 
    END AS esfera,
    CASE 
      WHEN natureza_juridica IN ('1015','1023', '1031','1104','1139','1112', '1147','1236','1120', '1155','1180','1341', '1163','1252','1260','1279','1287','1295','1309','1317','1325','1333','1171','1180', '1244', '1341') THEN 'Executivo'
      WHEN natureza_juridica IN ('1040','1058', '1066') THEN 'Legislativo'
      WHEN natureza_juridica IN ('1074','1082') THEN 'Judiciário'
      ELSE 'Outros' 
    END AS poderes,
    COUNT(*) AS quantidade_vinculos
  FROM `basedosdados.br_me_rais.microdados_vinculos`  
  GROUP BY ano, sigla_uf, esfera, natureza_juridica
) subquery
WHERE esfera != 'Outros' and poderes != 'Outros'
ORDER BY esfera, ano, sigla_uf, poderes;


---116-3 - Órgão Público Autônomo Federal - executivo
--125-2 - Fundação Pública de Direito Privado Federal
--126-0 - Fundação Pública de Direito Privado Estadual ou do Distrito Federal
--127-9 - Fundação Pública de Direito Privado Municipal
--128-7 - Fundo Público da Administração Indireta Federal
--129-5 - Fundo Público da Administração Indireta Estadual ou do Distrito Federal
--130-9 - Fundo Público da Administração Indireta Municipal
--131-7 - Fundo Público da Administração Direta Federal
--132-5 - Fundo Público da Administração Direta Estadual ou do Distrito Federal
--133-3 - Fundo Público da Administração Direta Municipal
--117-1 - Órgão Público Autônomo Estadual ou do Distrito Federal - executivo
--118-0 - Órgão Público Autônomo Municipal - executivo
--124-4 - Município
--134-1 - União '''

In [ ]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')
df

Downloading: 100%|██████████|


,ano,sigla_uf,esfera,poderes,quantidade_vinculos
0,1995,AC,Estadual,Executivo,21811
1,1995,AC,Estadual,Executivo,1354
2,1995,AC,Estadual,Executivo,142
3,1995,AC,Estadual,Judiciário,663
4,1995,AC,Estadual,Legislativo,504
...,...,...,...,...,...
11180,2022,TO,Municipal,Executivo,24
11181,2022,TO,Municipal,Executivo,12361
11182,2022,TO,Municipal,Executivo,3960
11183,2022,TO,Municipal,Executivo,1385


In [8]:
df['esfera'].unique()

array(['Estadual', 'Federal', 'Municipal'], dtype=object)

Subindo para o GBQ

In [11]:
client = bigquery.Client()
dataset_ref = client.dataset('perfil_remuneracao')

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11185 entries, 0 to 11184
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ano                  11185 non-null  Int64 
 1   sigla_uf             11185 non-null  object
 2   esfera               11185 non-null  object
 3   poderes              11185 non-null  object
 4   quantidade_vinculos  11185 non-null  Int64 
dtypes: Int64(2), object(3)
memory usage: 458.9+ KB


In [16]:
schema = [bigquery.SchemaField('ano', 'INTEGER', description= 'Ano de referência'),
          bigquery.SchemaField('quantidade_vinculos', 'INTEGER', description= 'Número total de vinculos observados'),
          bigquery.SchemaField('sigla_uf', 'STRING', description= 'Sigla da Unidade da Federação'),
          bigquery.SchemaField('esfera', 'STRING', description= 'Nível da esfera do governo'),
          bigquery.SchemaField('poderes', 'STRING', description= 'Poder abrangente ao nível de esfera')
          ]

In [17]:
table_ref = dataset_ref.table('RAIS_evolucao_esfera_poder')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=046144a1-91ac-4751-9166-921cdd81ae42>